# Neural Networks
__MATH 3480__ - Dr. Michael Olson

Reading:
* Geron, Chapter 5
* Geron, Chapter 10

## History

* The concept of a neural network was first theorized in 1943
* Technique of the Perceptron was introduced in 1958 by Frank Rosenblatt
  * Belief that ANNs (Artificial Neural Networks) would soon be able to translate languages on the fly
* Due to lack in technology and in data, funding failed and focuses shifted to other methods
  * "AI Winter"
* ImageNet 2012 - Large Scale Visual Recognition Challenge
  * Reintroduced Neural Networks
 
Today, systems are using ANNs everywhere (e.g. Apple and Google for speech and image recognition).

## The Perceptron

In a one-layer neural network, also known as a __perceptron__, only does one calculation. There are two parts:
* An input layer
* An output layer (where the calculation happens)

The value of the perceptron is some function of the input layer. For example,
$$y=x_1 + x_2 + \dots + x_n$$

<img src="https://github.com/drolsonmi/math3480/blob/main/Notes/Images/NN_01Perceptron.png?raw=true" width=350>

In [2]:
x1 = np.array([5,13])
x2 = np.array([4,1])
x1,x2

(array([ 5, 13]), array([4, 1]))

Sometimes, the value is not a direct sum. It is often a weighted sum, and the goal of the perceptron is to find the right weights.
$$y = w_1x_1 + w_2x_2 + \dots + w_nx_n$$

<img src="https://github.com/drolsonmi/math3480/blob/main/Notes/Images/NN_02PerceptronWeighted.png?raw=true" width=350>

But what if all $x_i=0$? Then the weights won't do anything and the perceptron becomes dead or useless. So, to prevent a zero value, we have to add in a bias term. This bias term can be positive or negative, and it can be large or small.
$$y = w_1x_1 + w_2x_2 + \dots + w_nx_n + b$$
$$y = \sum_i w_i x_i + b$$

<img src="https://github.com/drolsonmi/math3480/blob/main/Notes/Images/NN_03LinReg.png?raw=true" width=350>

<font color=red>__INSERT SECTION ON MLPs (MULTILAYER PERCEPTRONS)__</font>

Notice how this is a lot like a linear regression. It is just a version of,
$$AX=Y \qquad \left[a_1,a_2,\dots,a_n\right]\begin{bmatrix}
\vdots & \vdots & & \vdots \\
x_1 & x_2 & \dots & x_n \\
\vdots & \vdots & & \vdots
\end{bmatrix}=\left[y_1,y_2,\dots,y_n\right]$$

<img src="https://github.com/drolsonmi/math3480/blob/main/Notes/Images/NN_04MLP.png?raw=true" width=350>

<img src="https://github.com/drolsonmi/math3480/blob/main/Notes/Images/NN_05MLP2.png?raw=true" width=350>

So, at this point, this really is just a simple linear algebra problem, which we could solve using the Pseudoinverse:
$$A=YX^\dagger$$

This is the basic idea of the function of a perceptron. The result would be the value, or __activation__, of the output neuron(s).

In [ ]:
#####   LOAD THE DATA   #####
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data#[:, (2,3)] # petal length, petal width
#y = (iris.target == 0).astype(np.int) # Iris setosa
y = iris.target

#####   CROSS VALIDATION   #####
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=20)

#####   TRAIN THE PERCEPTRON MODEL   #####
from sklearn.linear_model import Perceptron
per_clf = Perceptron()
per_clf.fit(X_train,y_train)

#####   PREDICT FOR THE TEST DATA   #####
y_pred = per_clf.predict(X_test)
y_pred

#####   PLOT   #####
#import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

fig = plt.figure(figsize=(10,5))

ax1 = fig.add_subplot(121)
X_df = pd.DataFrame(X, columns=['Sepal Length','Sepal Width','Petal Length','Petal Width'])
X_df['Species'] = pd.DataFrame(y)
sns.scatterplot(data=X_df, x='Petal Length', y='Petal Width', hue='Species', ax=ax1)
ax1.set_title('Original Data')

ax2 = fig.add_subplot(122)
Xtest_df = pd.DataFrame(X_test, columns=['Sepal Length','Sepal Width','Petal Length','Petal Width'])
Xtest_df['Species'] = pd.DataFrame(y_pred)
sns.scatterplot(data=Xtest_df, x='Petal Length', y='Petal Width', hue='Species', ax=ax2)
ax2.set_title('Testing Data')

plt.show()

#####   EVALUATION   #####
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Two issues:
1. Need more data to train the model (80% * 150 = 120)
2. This is basically a multi-linear regression - we can do better

## Activation Functions

Now, what do we want the output of our activated neuron to look like?
* A value (regression)
* A 1 or a 0 (hard classification)
* A probability (soft classification)

To get the activation node to the activated value we want, we are going to take our weighted calculations and send them through a specific kind of function which we call an __activation function__.

$$\hat{y} = \sigma(z) \qquad z = \sum_i w_ix_i + b$$

The type of activation function we use depends on the need. Up until now, we have only dealt with a linear function.

#### Linear function
$$\hat{y} = \sigma(z) = z = \sum_i w_ix_i + b$$

However, activation functions are not always linear. Often, we want it to be a number between 0 and 1. Here are a few nonlinear activation functions:

#### Step function
Also known as the *Threshold Logic Unit (TLU)* or *Linear Threshold Unity (LTU)*

Utilizes $H(z)$, the Heaviside Step Function

$$\hat{y} = \sigma(z) = H(z) = \left\{\begin{align*}1 & \text{ if }z \ge 0 \\ 0 & \text{ if }z < 0\end{align*}\right. \qquad z = \sum_i w_ix_i + b$$

![Step Curve](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4b/Activation_binary_step.svg/120px-Activation_binary_step.svg.png)

#### Sigmoid function (or Logical function)
  * More sensitive to small changes
$$\hat{y} = \sigma(z) = \frac{1}{1+e^{-z}} \qquad z = \sum_i w_ix_i + b$$
![Sigmoid Curve](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5b/Activation_logistic.svg/120px-Activation_logistic.svg.png)

#### Hyperbolic Tangent
  * This one acts like the Sigmoid function, but goes from -1 to 1
$$\tanh(z)=\frac{e^z - e^{-z}}{e^z + e^{-z}} \qquad z = \sum_i w_ix_i + b$$
![Hyperbolic Tangent Curve](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Activation_tanh.svg/120px-Activation_tanh.svg.png)

#### Rectified Linear Unit (ReLU)
  $$y=\max\{0,z\} \qquad z = \sum_i w_ix_i + b$$
![ReLU Curve](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Activation_rectified_linear.svg/120px-Activation_rectified_linear.svg.png)

Commonly used in hidden layers to solve the *vanishing gradient* problem.
* Discuss more later when we talk about traning neural networks

There are many other activation functions. However, in order to train our network, our activation function needs to be differentiable.

## Hidden Layers

Knowing the basic perceptron, we can solve basic problems. But we can do little better than an ordinary linear regression. We can improve performance by adding a second or third step before getting our prediction $\hat{y}$. These extra layers between the input layer and output layer are called __hidden layers__, and a basic neural network with 1 or more hidden layers is a __multi-layer perceptron__, a basic __artificial neural network__.

### Some basic terminology
* The __width of a layer__ is the number of neurons within that layer
* The __depth of a network__ is the number of (*Hidden?*) layers in a network
* An ANN becomes a __Deep Neural Network__ (DNN) when there are 2 or more hidden layers

## Training an ANN

An ANN most often doesn't get it right the first time. The program has to iterate through the network multiple times to get it right. Each iteration updates the activators (neurons) to improve the result. The activators are updated with new weights in a variety of ways.

The most common way to update weights in an activator is via gradient descent:
$$w_{ij}'=w_{ij} + \eta\left(y_i - \hat{y}_i\right)x_i$$
where
* $w_{ij}$ is the current weight, and $w_{ij}'$ is the updated weight
* $\eta$ is the learning rate
* $x_i$ is the input
* $y_i$ is the target output
* $\hat{y}_i$ is the actual output using the weight of $w_{ij}$

This is the algorithm developed by Frank Rosenblatt in 1958, inspired by *Hebb's Rule*, which was published in 1949. 

#### More on ReLU Activation Function
When we have a DNN, the gradient descent algorithm has to be performed at each stage. However, this poses a problem. The *backpropagation* method of training a DNN (think very intense process of doing gradient descent) uses the chain rule to capture errors between steps. The chain rule requires us to multiply our derivatives together, which is where the problem arises.

Imagine you have a derivative of 0.6 at each of 7 steps. This indicates a large error. But because of the chain rule, we multiply these together and we get $0.6^7 = 0.028$, which is a small error.

To prevent this, we limit how low we can go. The ReLU activation function addresses this issue.

## ANNs Today

The result of these neural networks are quite good.
* [Mona Lisa can talk!](https://www.youtube.com/watch?v=P2uZF-5F1wI)
* [Mark Rober - Stealing Baseball Signs with a Phone](https://www.youtube.com/watch?v=PmlRbfSavbI)

Are Neural Networks perfect yet? No...
* [Neurabites.com](https://neurabites.com/muffin-or-chihuahua/)

### Cautions with ANNs

* ANNs are relatively easy to train. However, because it's so easy, many people are just using ANNs without understanding what is happening under the hood.

Some of the issues:
* Easy to overfit
  * There are some cases where we want very specific results, but it's very easy to overfit
* Not very good when we want to generalize things
  * Not very good with learning physics (model it to predict where a cannonball lands, but it can't use that to launch a rocket)
* Interpretable/Explainable
  * The number of neurons adds a large number of degrees of freedom which makes it very expressive
  * More degrees of freedom makes it more complicated and harder to interpret
* Incorporating Physics
  * It is a difficult process to incorporate our knowledge of the real world
  * True for all ML Algorithms, but especially for ANNs.

## Building an ANN

To build an ANN, we first create our input and output layers.

### Input
How many datapoints are going into the model? 

In the MNIST dataset, each image had a resolution of 28x28, or a total of 28*28=784 pixels. So, our input layer has a width of 784, one node for each pixel.

### Output
What is the desired output? How many nodes are needed to represent that output?

In the MNIST dataset, the images are of numerical digits. So, the output is going to be any of the 10 digit. If we create 10 nodes (one for each digit), then we can use a Logistic/Softmax activation function to get a probability between 0 and 1. So, our output layer will have a width of 10.

### Hidden layers
Now, we determine the hidden layers. We have to decide how many hidden layers are needed and the width of each hidden layer.

#### Number of hidden layers
One hidden layer is generally enough, but deep neural networks (more than one hidden layer) have a higher parameter efficiency. Deep networks can use exponentially fewer neurons than shallow networks. Some rules of thumb:
* lower hidden layers (layers near the input) model low-level structures (e.g., line segments of various shapes and orientations)
* intermediate hidden layers combine these low-level structures to model intermediate-level structures (e.g., squares, circles)
* Higher hidden layers (layers near the output) model high-level structures (e.g., faces)

*Transfer learning*: using lower layers from one model in another situation. For example, using the lower layers in the above example to recognize line segments, then build a network to identify animals instead of faces.

#### Width of hidden layers
Early models used a pyramid structure - larger layers leading to gradually smaller layers. Experience has shown this really doesn't perform any better than layers with the same number of neurons. In fact, equally-sized layers tend to perform slightly better than decreasing layers.

Generally, we do start with a larger first hidden layer, then shrink to a smaller layer and keep other hidden layers roughly the same size.

In the past, programmers would retrain NNs, gradually increasing the number of neurons, until the model starts to overfit. More modern models start with large numbers of neurons and use early-stopping techniques to prevent the models from overfitting.

## Example with Fashion MNIST dataset

* Create a new virtual environment for Tensorflow and Keras
* Install Tensorflow and Keras
* Check for the version of Tensorflow and Keras that you are using

In [ ]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

#### Load the data

In [ ]:
# Load the dataset
fashion_mnist = keras.datasets.fashion_mnist
(X, y), (X_test, y_test) = fashion_mnist.load_data()

# There are 60,000 images of size 28x28
X.shape

In [ ]:
X_test.shape

In [ ]:
# NOTE! Each pixel of the image is represended as a value from 0 to 255.
# Two problems
   # We want a value from 0 to 1
   # It is an integer, not a float value
# To fix both, divide by 255.0


# We have a test set, but we need a validation set:
X_valid, X_train = X[:5000] / 255.0 , X[5000:] / 255.0
y_valid, y_train = y[:5000], y[5000:]
X_test = X_test / 255.0

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
[class_names[y_train[i]] for i in range(20) ]

#### Plot the training images

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt

ax = ['ax1','ax2','ax3','ax4','ax5','ax6','ax7','ax8','ax9','ax10','ax11','ax12','ax13','ax14','ax15','ax16','ax17','ax18','ax19','ax20']
f, ax = plt.subplots(1, 20, sharey=True, figsize=(25,6))
for i in range(20):
    img = ax[i].imshow(X_train[i])
    cmap = plt.cm.get_cmap('gray_r')
    img.set_cmap(cmap)
    ax[i].axis('off')

#### Create the model

In [ ]:
model = keras.models.Sequential()                        # Input layer - Single stack of layers
model.add(keras.layers.Flatten(input_shape=[28,28]))     # Some Preprocessing - converts each input image into a 1D array
model.add(keras.layers.Dense(300, activation="relu"))    # Hidden layer
model.add(keras.layers.Dense(100, activation="relu"))    # Hidden layer
model.add(keras.layers.Dense(10, activation="softmax"))  # Output layer

In [ ]:
# This cell is the same as the previous cell

from keras.layers import Dense
model = keras.models.Sequential([                        # Input layer - Single stack of layers
    keras.layers.Flatten(input_shape=[28,28]),           # Some Preprocessing - converts each input image into a 1D array
    Dense(300, activation="relu"),                       # Hidden layer
    Dense(100, activation="relu"),                       # Hidden layer
    Dense(10, activation="softmax")                      # Output layer
])

In [ ]:
model.summary()
# "Param #" is the number of weights and biases leading into that layer
# The first layer has 784 neurons
# That is 784*300 connections, so 784*300 weights and 300 biases
# total of 235200 + 300 = 235500

In [ ]:
# Get a list of the layers
model.layers

In [ ]:
# Get values for all weights leading to a layer
weights, biases = model.layers[1].get_weights()
weights

In [ ]:
biases

#### Train the model

In [ ]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",                           # sgd = Stochastic Gradient Descent
              metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs = 30, validation_data=(X_valid, y_valid))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

#### Test the model

In [ ]:
ax = ['ax1','ax2','ax3']
f, ax = plt.subplots(1, 3, sharey=True, figsize=(6,6))
for i in range(3):
    img = ax[i].imshow(X_test[i])
    cmap = plt.cm.get_cmap('gray_r')
    img.set_cmap(cmap)
    ax[i].axis('off')

In [ ]:
### Classes:  
# ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
y_prob = model.predict(X_test[:3])
y_prob.round(2)

In [ ]:
# Get values for all weights leading to a layer
weights, biases = model.layers[1].get_weights()
weights

In [ ]:
biases

## Saving the model

In [ ]:
# To save a model for future use
model.save("FashionMNIST.h5")

In [ ]:
# To load a model that was previously saved
loaded_model = keras.models.load_model('FashionMNIST.h5')

# Then, use as normal
starting_image = 17
y_prob = loaded_model.predict(X_test[starting_image:starting_image+3])
print(y_prob.round(2))

f, ax = plt.subplots(1, 3, sharey=True, figsize=(6,6))
for i in range(3):
    img = ax[i].imshow(X_test[starting_image+i])
    cmap = plt.cm.get_cmap('gray_r')
    img.set_cmap(cmap)
    ax[i].axis('off')

### Classes:  
# ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]